In [1]:
import pyodbc 

conn_str = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=PaymentData;'
    r'Trusted_Connection=yes;'
    r'Encrypt=no;'
    )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

In [ ]:
cursor.close()
cnxn.close()

Для начала извлечем случайные `Oid` для Bank, Supplier, Client, Cashbox, Project среди тех записей, которые участвуют в платежах (табл. PaymentParticipant).

In [2]:
def get_random_oid(table_name):
    query = f"""
    SELECT TOP 1 {table_name}.Oid
    FROM {table_name} 
    INNER JOIN PaymentParticipant ON {table_name}.Oid = PaymentParticipant.Oid
    ORDER BY NEWID()
    """
    
    oid = cursor.execute(query).fetchval()
    print(f"Selected Oid for table {table_name}: {oid}")
    return oid


def get_project_oid_by_client_oid(client_oid):
    query = f"""
    SELECT Oid
	FROM Project
	WHERE Client = '{client_oid}'
    """

    oid = cursor.execute(query).fetchval()
    print(f"Project Oid: {oid}")
    return oid

In [3]:
bank_oid = get_random_oid("Bank")
supplier_oid = get_random_oid("Supplier")
client_oid = get_random_oid("Client")
cashbox_oid = get_random_oid("Cashbox")
project_oid = get_project_oid_by_client_oid(client_oid)

Selected Oid for table Bank: 775A7217-D568-DA9F-9DA4-FB9CE0657E75
Selected Oid for table Supplier: 92CCB55D-9845-3301-4E96-904638524379
Selected Oid for table Client: 2CB9AF02-96CD-F6FE-C857-6F69A37A33A5
Selected Oid for table Cashbox: D6827B8F-D516-0111-3553-34CB56E9BEFA
Project Oid: 37C4308E-80A9-8DC2-E263-C7A2F1B7CFEB


Функция для получения баланса объектов

In [4]:
def get_balance_by_oid(oid, object_name=None):
    query = f"""
    SELECT Balance FROM PaymentParticipant 
    WHERE Oid = '{oid}'
    """

    balance = cursor.execute(query).fetchval()
    if object_name is not None:
        print(f"{object_name} balance: {balance}")
    return balance
    

Посмотрим на балансы объектов в начале

In [5]:
client_balance = get_balance_by_oid(client_oid, "Client")
bank_balance = get_balance_by_oid(bank_oid, "Bank")
cashbox_balance = get_balance_by_oid(cashbox_oid, "Cashbox")
supplier_balance = get_balance_by_oid(supplier_oid, "Supplier")

Client balance: 10803
Bank balance: -10410
Cashbox balance: -33651
Supplier balance: -8267


In [6]:
client_balance - 50000

-39197

In [7]:
bank_balance - 300000

-310410

In [8]:
cashbox_balance + 50000

16349

In [9]:
supplier_balance + 300000

291733

In [21]:
client_balance = get_balance_by_oid(client_oid, "Client")
bank_balance = get_balance_by_oid(bank_oid, "Bank")
cashbox_balance = get_balance_by_oid(cashbox_oid, "Cashbox")
supplier_balance = get_balance_by_oid(supplier_oid, "Supplier")

Client balance: -39197
Bank balance: -310410
Cashbox balance: 16349
Supplier balance: 291733


Новые платежи проведем как в файле _Balance description.pdf_

Для вставки новых платежей необходимо знать `Oid` категории платежа, функция для этого:

In [11]:
def get_payment_category_oid(payment_name):
    query = f"""
    SELECT Oid FROM PaymentCategory
    WHERE Name = '{payment_name}'
    """ 
    
    category_oid = cursor.execute(query).fetchval()
    return category_oid

Согласно примеру из таблицы есть 4 платежа:

1. `Bank` переводит `Supplier` авансовый платеж **400_000** на приобретение материалов.

    `Bank balance -400_000`, `Cashbox balance 0`, `Client balance 0`, `Supplier balance +400_000`. 

2. `Supplier` переводит `Client` **100_000** для закупки материалов

    `Bank balance -400_000`, `Cashbox balance 0`, `Client balance +100_000`, `Supplier balance +300_000`. 

3. `Client` передает в `Cashbox` **150_000** для закупки материалов

    `Bank balance -400_000`, `Cashbox balance +150_000`, `Client balance -50_000`, `Supplier balance +300_000`. 

4. `Cashbox` частично гасит кредит в `Bank` суммой в **100_000**
   
   `Bank balance -300_000`, `Cashbox balance +50_000`, `Client balance -50_000`, `Supplier balance +300_000`.

Указаны относительные (к предыдущему платежу) изменения балансов, сохраним их для каждого платежа

In [13]:
BALANCE_CHANGES = {
    "First payment": {"Bank": -400_000, "Cashbox": 0, "Client": 0, "Supplier": 400_000},
    "Second payment": {"Bank": 0, "Cashbox": 0, "Client": 100_000, "Supplier": -100_000},
    "Third payment": {"Bank": 0, "Cashbox": 150_000, "Client": -150_000, "Supplier": 0},
    "Fourth payment": {"Bank": 100_000, "Cashbox": -100_000, "Client": 0, "Supplier": 0},
}

Создадим для внесения платежей соответствующие функции

In [14]:
def make_first_payment():
    payment_category_oid = get_payment_category_oid("Аванс на материалы")
    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 400000, '{payment_category_oid}', '{project_oid}', NULL,
    'Авансовый платеж поставщику', NULL, '{bank_oid}', '{supplier_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
    cursor.commit()

def make_second_payment():
    payment_category_oid = get_payment_category_oid("Закупка материалов")
    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 100000, '{payment_category_oid}', '{project_oid}', NULL,
    'Для клиента приобрели материал', NULL, '{supplier_oid}', '{client_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
    cursor.commit()

def make_third_payment():
    payment_category_oid = get_payment_category_oid("Закупка материалов")
    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 150000, '{payment_category_oid}', '{project_oid}', NULL,
    'Для клиента приобрели материал', NULL, '{client_oid}', '{cashbox_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
    cursor.commit()

def make_fourth_payment():
    payment_category_oid = get_payment_category_oid("Возврат кредита")
    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
	Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
	CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), 100000, '{payment_category_oid}', '{project_oid}', NULL,
    'Частичное погашение кредита ', NULL, '{cashbox_oid}', '{bank_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
    cursor.commit()


In [15]:
def assert_after_payment(payment, object, object_oid, old_object_balance):
    current_balance = get_balance_by_oid(object_oid)
    balance_change = BALANCE_CHANGES[payment][object]
    expected_balance = old_object_balance + balance_change
    assert_msg = f'{object} balance {current_balance}, expected {expected_balance}'
    assert current_balance == expected_balance, assert_msg
    print(assert_msg)

Первый платеж (Bank -> Supplier)

In [16]:
bank_balance_before_payment = get_balance_by_oid(bank_oid)
cashbox_balance_before_payment = get_balance_by_oid(cashbox_oid)
client_balance_before_payment = get_balance_by_oid(client_oid)
supplier_balance_before_payment = get_balance_by_oid(supplier_oid)

make_first_payment()

assert_after_payment("First payment", "Bank", bank_oid, bank_balance_before_payment)
assert_after_payment("First payment", "Cashbox", cashbox_oid, cashbox_balance_before_payment)
assert_after_payment("First payment", "Client", client_oid, client_balance_before_payment)
assert_after_payment("First payment", "Supplier", supplier_oid, supplier_balance_before_payment)

Bank balance -410410, expected -410410
Cashbox balance -33651, expected -33651
Client balance 10803, expected 10803
Supplier balance 391733, expected 391733


Второй платеж (Supplier -> Client)

In [17]:
bank_balance_before_payment = get_balance_by_oid(bank_oid)
cashbox_balance_before_payment = get_balance_by_oid(cashbox_oid)
client_balance_before_payment = get_balance_by_oid(client_oid)
supplier_balance_before_payment = get_balance_by_oid(supplier_oid)

make_second_payment()

assert_after_payment("Second payment", "Bank", bank_oid, bank_balance_before_payment)
assert_after_payment("Second payment", "Cashbox", cashbox_oid, cashbox_balance_before_payment)
assert_after_payment("Second payment", "Client", client_oid, client_balance_before_payment)
assert_after_payment("Second payment", "Supplier", supplier_oid, supplier_balance_before_payment)

Bank balance -410410, expected -410410
Cashbox balance -33651, expected -33651
Client balance 110803, expected 110803
Supplier balance 291733, expected 291733


Третий платеж (Client -> Cashbox)

In [18]:
bank_balance_before_payment = get_balance_by_oid(bank_oid)
cashbox_balance_before_payment = get_balance_by_oid(cashbox_oid)
client_balance_before_payment = get_balance_by_oid(client_oid)
supplier_balance_before_payment = get_balance_by_oid(supplier_oid)

make_third_payment()

assert_after_payment("Third payment", "Bank", bank_oid, bank_balance_before_payment)
assert_after_payment("Third payment", "Cashbox", cashbox_oid, cashbox_balance_before_payment)
assert_after_payment("Third payment", "Client", client_oid, client_balance_before_payment)
assert_after_payment("Third payment", "Supplier", supplier_oid, supplier_balance_before_payment)

Bank balance -410410, expected -410410
Cashbox balance 116349, expected 116349
Client balance -39197, expected -39197
Supplier balance 291733, expected 291733


Четвертый платеж (Cashbox -> Bank)

In [20]:
bank_balance_before_payment = get_balance_by_oid(bank_oid)
cashbox_balance_before_payment = get_balance_by_oid(cashbox_oid)
client_balance_before_payment = get_balance_by_oid(client_oid)
supplier_balance_before_payment = get_balance_by_oid(supplier_oid)

make_fourth_payment()

assert_after_payment("Fourth payment", "Bank", bank_oid, bank_balance_before_payment)
assert_after_payment("Fourth payment", "Cashbox", cashbox_oid, cashbox_balance_before_payment)
assert_after_payment("Fourth payment", "Client", client_oid, client_balance_before_payment)
assert_after_payment("Fourth payment", "Supplier", supplier_oid, supplier_balance_before_payment)

Bank balance -310410, expected -310410
Cashbox balance 16349, expected 16349
Client balance -39197, expected -39197
Supplier balance 291733, expected 291733
